# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.113558e+02     1.736299e+00
 * time: 0.12709999084472656
     1     1.093049e+01     9.390822e-01
 * time: 0.924921989440918
     2    -1.137047e+01     1.089996e+00
 * time: 1.0521411895751953
     3    -3.368680e+01     8.827084e-01
 * time: 1.2136061191558838
     4    -4.701434e+01     7.297427e-01
 * time: 1.3607730865478516
     5    -5.648289e+01     2.951362e-01
 * time: 1.5039150714874268
     6    -5.923424e+01     2.958215e-01
 * time: 1.6086421012878418
     7    -6.047148e+01     1.239297e-01
 * time: 1.7209131717681885
     8    -6.106686e+01     5.136064e-02
 * time: 1.825890064239502
     9    -6.139974e+01     4.800026e-02
 * time: 1.9380011558532715
    10    -6.169749e+01     3.795920e-02
 * time: 2.0426480770111084
    11    -6.188527e+01     2.881671e-02
 * time: 2.1544041633605957
    12    -6.201471e+01     1.947802e-02
 * time: 2.2590761184692383
    13    -6.206966e+01     1.604567e-02
 * time: 2.37086820602

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557678
    AtomicNonlocal      14.8522646
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336821

    total               -62.261666462299
